In [35]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Dev\amo-personal-project`


In [36]:
# Pkg.add("JuMP")
# Pkg.add("HiGHS")
# Pkg.add("Plots")
using JuMP, HiGHS, Plots

In [37]:
# Define all sets
Ω = collect(1:2)
T = collect(1:2) # time periods
n_N = 24 # number of nodes
N = collect(1:n_N) # nodes
G = collect(1:16) # generators
L = collect(1:34) # transmission lines

34-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
  ⋮
 26
 27
 28
 29
 30
 31
 32
 33
 34

In [38]:
# Define scenario probabilities
π_ω = [0.5,0.5]

2-element Vector{Float64}:
 0.5
 0.5

In [39]:
# Define connections between nodes as an upper triangular matrix
orig_n = [1, 1, 1, 2, 2, 3,  3, 4,  5,  6, 7, 8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 14, 15, 15, 15, 16, 16, 17, 17, 18, 19, 20, 21]
dest_n = [2, 3, 5, 4, 6, 9, 24, 9, 10, 10, 8, 9, 10, 11, 12, 11, 12, 13, 14, 13, 23, 23, 16, 16, 21, 24, 17, 19, 18, 22, 21, 20, 23, 22]

lines = zeros(n_N, n_N)

for i in 1:size(orig_n)[1]
    lines[orig_n[i], dest_n[i]] = i # incoming transmission lines
    lines[dest_n[i], orig_n[i]] = -i # outgoing transmission lines
end
lines

24×24 Matrix{Float64}:
  0.0   1.0   2.0   0.0   3.0    0.0    0.0  …    0.0    0.0   0.0   0.0  0.0
 -1.0   0.0   0.0   4.0   0.0    5.0    0.0       0.0    0.0   0.0   0.0  0.0
 -2.0   0.0   0.0   0.0   0.0    0.0    0.0       0.0    0.0   0.0   0.0  7.0
  0.0  -4.0   0.0   0.0   0.0    0.0    0.0       0.0    0.0   0.0   0.0  0.0
 -3.0   0.0   0.0   0.0   0.0    0.0    0.0       0.0    0.0   0.0   0.0  0.0
  0.0  -5.0   0.0   0.0   0.0    0.0    0.0  …    0.0    0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0    0.0    0.0       0.0    0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0    0.0  -11.0       0.0    0.0   0.0   0.0  0.0
  0.0   0.0  -6.0  -8.0   0.0    0.0    0.0       0.0    0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0  -9.0  -10.0    0.0       0.0    0.0   0.0   0.0  0.0
  ⋮                              ⋮           ⋱           ⋮                
  0.0   0.0   0.0   0.0   0.0    0.0    0.0  …    0.0    0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0    0.0    0.0 

In [40]:
function get_lines_for_node(node, incoming)
    if incoming
        return filter(i -> i >= 1.0, lines[node, 1:end])
    end
    return filter(i -> i <= -1.0, lines[node, 1:end])
end

get_lines_for_node (generic function with 1 method)

In [41]:
# Parameters
c = [13.32, 13.32, 20.7, 20.93, 26.11, 10.52, 10.52, 6.02, 5.47, 7, 10.52, 10.89, 0, 0, 0, 0]
P_max = [ # maximum generation capacity of each generator
    # convetional generators
    [[106.4, 106.4],[106.4, 106.4]],
    [[106.4, 106.4],[106.4, 106.4]],
    [[245, 245],    [245, 245]],
    [[413.7, 413.7],[413.7, 413.7]],
    [[42, 42],      [42, 42]],
    [[108.5, 108.5],[108.5, 108.5]],
    [[108.5, 108.5],[108.5, 108.5]],
    [[280, 280],    [280, 280]],
    [[280, 280],    [280, 280]],
    [[210, 210],    [210, 210]],
    [[217, 217],    [217, 217]],
    [[245, 245],    [245, 245]],
     # renewable generators
    [[500, 500],    [500, 500]],
    [[500, 500],    [500, 500]],
    [[500, 500],    [500, 500]],
    [[500, 500],    [500, 500]]
] 
# transmission line capacity
F_max = [175, 175, 350, 175, 175, 175, 400, 175, 350, 175, 350, 175, 175, 400, 400, 400, 400,
        500, 500, 500, 500, 250, 250, 500, 400, 500, 500, 500, 500, 500, 1000, 1000, 1000, 500]
# battery capacity
B_max = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# demand 
D = [84, 75, 139, 58, 55, 106, 97, 132, 135, 150, 0, 0, 205, 150, 245, 77, 0, 258, 141, 100, 0, 0, 0, 0]

24-element Vector{Int64}:
  84
  75
 139
  58
  55
 106
  97
 132
 135
 150
   ⋮
  77
   0
 258
 141
 100
   0
   0
   0
   0

In [42]:
# Location of the generators
loc_G = [1, 2, 7, 13, 15, 15, 16, 18, 21, 22, 23, 23, 3, 5, 16, 21]

16-element Vector{Int64}:
  1
  2
  7
 13
 15
 15
 16
 18
 21
 22
 23
 23
  3
  5
 16
 21

In [43]:
function get_generator_for_node(node)
    return findall(i -> i == node, loc_G)
end

get_generator_for_node (generic function with 1 method)

In [44]:
model = Model(HiGHS.Optimizer)

@variable(model, P_max[g][t][ω] >= P[g in G, t in T, ω in Ω] >= 0) 
@variable(model, F_max[l] >= F[l in L, t in T, ω in Ω] >= -F_max[l])

@variable(model, B_max[n] >= B[n in N, t in vcat([0],T), ω in Ω] >= 0) 
@variable(model, ΔB[n in N, t in T, ω in Ω])

@constraint(model, init_battery_charge[n in N, ω in Ω], B[n,0,ω] == 0)
@constraint(model, battery_charge[n in N, t in T, ω in Ω], B[n,t-1,ω] + ΔB[n,t,ω] == B[n,t,ω])

@constraint(model, power_flow[n in N, t in T, ω in Ω], 
    sum(P[g,t,ω] for g in get_generator_for_node(n))
    - D[n]
    + sum(F[i,t,ω] for i in get_lines_for_node(n, true))
    - sum(F[-i,t,ω] for i in get_lines_for_node(n, false))
    - ΔB[n,t,ω]
    == 0
)

@objective(model, Min, sum(π_ω[ω]*c[g]*P[g,t,ω] for g in G, t in T, ω in Ω))

print(model)

Min 6.66 P[1,1,1] + 6.66 P[1,1,2] + 6.66 P[1,2,1] + 6.66 P[1,2,2] + 6.66 P[2,1,1] + 6.66 P[2,1,2] + 6.66 P[2,2,1] + 6.66 P[2,2,2] + 10.35 P[3,1,1] + 10.35 P[3,1,2] + 10.35 P[3,2,1] + 10.35 P[3,2,2] + 10.465 P[4,1,1] + 10.465 P[4,1,2] + 10.465 P[4,2,1] + 10.465 P[4,2,2] + 13.055 P[5,1,1] + 13.055 P[5,1,2] + 13.055 P[5,2,1] + 13.055 P[5,2,2] + 5.26 P[6,1,1] + 5.26 P[6,1,2] + 5.26 P[6,2,1] + 5.26 P[6,2,2] + 5.26 P[7,1,1] + 5.26 P[7,1,2] + 5.26 P[7,2,1] + 5.26 P[7,2,2] + 3.01 P[8,1,1] + 3.01 P[8,1,2] + 3.01 P[8,2,1] + 3.01 P[8,2,2] + 2.735 P[9,1,1] + 2.735 P[9,1,2] + 2.735 P[9,2,1] + 2.735 P[9,2,2] + 3.5 P[10,1,1] + 3.5 P[10,1,2] + 3.5 P[10,2,1] + 3.5 P[10,2,2] + 5.26 P[11,1,1] + 5.26 P[11,1,2] + 5.26 P[11,2,1] + 5.26 P[11,2,2] + 5.445 P[12,1,1] + 5.445 P[12,1,2] + 5.445 P[12,2,1] + 5.445 P[12,2,2]
Subject to


 init_battery_charge[1,1] : B[1,0,1] == 0.0
 init_battery_charge[2,1] : B[2,0,1] == 0.0
 init_battery_charge[3,1] : B[3,0,1] == 0.0
 init_battery_charge[4,1] : B[4,0,1] == 0.0
 init_battery_charge[5,1] : B[5,0,1] == 0.0
 init_battery_charge[6,1] : B[6,0,1] == 0.0
 init_battery_charge[7,1] : B[7,0,1] == 0.0
 init_battery_charge[8,1] : B[8,0,1] == 0.0
 init_battery_charge[9,1] : B[9,0,1] == 0.0
 init_battery_charge[10,1] : B[10,0,1] == 0.0
 init_battery_charge[11,1] : B[11,0,1] == 0.0
 init_battery_charge[12,1] : B[12,0,1] == 0.0
 init_battery_charge[13,1] : B[13,0,1] == 0.0
 init_battery_charge[14,1] : B[14,0,1] == 0.0
 init_battery_charge[15,1] : B[15,0,1] == 0.0
 init_battery_charge[16,1] : B[16,0,1] == 0.0
 init_battery_charge[17,1] : B[17,0,1] == 0.0
 init_battery_charge[18,1] : B[18,0,1] == 0.0
 init_battery_charge[19,1] : B[19,0,1] == 0.0
 init_battery_charge[20,1] : B[20,0,1] == 0.0
 init_battery_charge[21,1] : B[21,0,1] == 0.0
 init_battery_charge[22,1] : B[22,0,1] == 0.0
 init


 power_flow[21,1,1] : P[9,1,1] + P[16,1,1] - F[25,1,1] - F[31,1,1] + F[34,1,1] - ΔB[21,1,1] == 0.0
 power_flow[22,1,1] : P[10,1,1] - F[30,1,1] - F[34,1,1] - ΔB[22,1,1] == 0.0
 power_flow[23,1,1] : P[11,1,1] + P[12,1,1] - F[21,1,1] - F[22,1,1] - F[33,1,1] - ΔB[23,1,1] == 0.0
 power_flow[24,1,1] : -F[7,1,1] - F[26,1,1] - ΔB[24,1,1] == 0.0
 power_flow[1,2,1] : P[1,2,1] + F[1,2,1] + F[2,2,1] + F[3,2,1] - ΔB[1,2,1] == 84.0
 power_flow[2,2,1] : P[2,2,1] - F[1,2,1] + F[4,2,1] + F[5,2,1] - ΔB[2,2,1] == 75.0
 power_flow[3,2,1] : P[13,2,1] - F[2,2,1] + F[6,2,1] + F[7,2,1] - ΔB[3,2,1] == 139.0
 power_flow[4,2,1] : -F[4,2,1] + F[8,2,1] - ΔB[4,2,1] == 58.0
 power_flow[5,2,1] : P[14,2,1] - F[3,2,1] + F[9,2,1] - ΔB[5,2,1] == 55.0
 power_flow[6,2,1] : -F[5,2,1] + F[10,2,1] - ΔB[6,2,1] == 106.0
 power_flow[7,2,1] : P[3,2,1] + F[11,2,1] - ΔB[7,2,1] == 97.0
 power_flow[8,2,1] : -F[11,2,1] + F[12,2,1] + F[13,2,1] - ΔB[8,2,1] == 132.0
 power_flow[9,2,1] : -F[6,2,1] - F[8,2,1] - F[12,2,1] + F[14,2,1] + F[1


 B[1,0,1] >= 0.0
 B[2,0,1] >= 0.0
 B[3,0,1] >= 0.0
 B[4,0,1] >= 0.0
 B[5,0,1] >= 0.0
 B[6,0,1] >= 0.0
 B[7,0,1] >= 0.0
 B[8,0,1] >= 0.0
 B[9,0,1] >= 0.0
 B[10,0,1] >= 0.0
 B[11,0,1] >= 0.0
 B[12,0,1] >= 0.0
 B[13,0,1] >= 0.0
 B[14,0,1] >= 0.0
 B[15,0,1] >= 0.0
 B[16,0,1] >= 0.0
 B[17,0,1] >= 0.0
 B[18,0,1] >= 0.0
 B[19,0,1] >= 0.0
 B[20,0,1] >= 0.0
 B[21,0,1] >= 0.0
 B[22,0,1] >= 0.0
 B[23,0,1] >= 0.0
 B[24,0,1] >= 0.0
 B[1,1,1] >= 0.0
 B[2,1,1] >= 0.0
 B[3,1,1] >= 0.0
 B[4,1,1] >= 0.0
 B[5,1,1] >= 0.0
 B[6,1,1] >= 0.0
 B[7,1,1] >= 0.0
 B[8,1,1] >= 0.0
 B[9,1,1] >= 0.0
 B[10,1,1] >= 0.0
 B[11,1,1] >= 0.0
 B[12,1,1] >= 0.0
 B[13,1,1] >= 0.0
 B[14,1,1] >= 0.0
 B[15,1,1] >= 0.0
 B[16,1,1] >= 0.0
 B[17,1,1] >= 0.0
 B[18,1,1] >= 0.0
 B[19,1,1] >= 0.0
 B[20,1,1] >= 0.0
 B[21,1,1] >= 0.0
 B[22,1,1] >= 0.0
 B[23,1,1] >= 0.0
 B[24,1,1] >= 0.0
 B[1,2,1] >= 0.0
 B[2,2,1] >= 0.0
 B[3,2,1] >= 0.0
 B[4,2,1] >= 0.0
 B[5,2,1] >= 0.0
 B[6,2,1] >= 0.0
 B[7,2,1] >= 0.0
 B[8,2,1] >= 0.0
 B[9,2,1] >= 0.0


In [45]:
optimize!(model)

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
192 rows, 296 cols, 528 nonzeros
64 rows, 168 cols, 272 nonzeros
64 rows, 168 cols, 272 nonzeros
Presolve : Reductions: rows 64(-176); columns 168(-272); elements 272(-496)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3939839171e-02 Pr: 64(33772) 0s
        114     2.2645800000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 114
Objective value     :  2.2645800000e+03
HiGHS run time      :          0.01


In [46]:
print(value.(P))

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    Dimension 2, [1, 2]
    Dimension 3, [1, 2]
And data, a 16×2×2 Array{Float64, 3}:
[:, :, 1] =
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
 207.0  207.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
 500.0  500.0
 500.0  500.0
 500.0  500.0
 500.0  500.0

[:, :, 2] =
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
 207.0  207.0
   0.0    0.0
   0.0    0.0
   0.0    0.0
 500.0  500.0
 500.0  500.0
 500.0  500.0
 500.0  500.0

In [47]:
print(value.(F))

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
    Dimension 2, [1, 2]
    Dimension 3, [1, 2]
And data, a 34×2×2 Array{Float64, 3}:
[:, :, 1] =
 -175.0  -175.0
   63.0   164.0
  196.0    95.0
 -169.0  -169.0
   69.0    69.0
 -175.0   -74.0
 -123.0  -123.0
 -111.0  -111.0
 -249.0  -350.0
  175.0   175.0
   97.0    97.0
  151.0    54.0
   78.0   175.0
  400.0   400.0
 -400.0  -396.0
  200.0  -250.0
  -46.0   400.0
  500.0    50.0
  100.0   100.0
 -455.0    -5.0
    9.0     9.0
  250.0   250.0
  250.0   250.0
  173.0   173.0
  -51.0   -51.0
  123.0   123.0
  500.0   500.0
 -500.0  -500.0
  500.0   500.0
   -0.0    -0.0
  758.0   758.0
 -359.0  -359.0
 -259.0  -259.0
   -0.0    -0.0

[:, :, 2] =
 -175.0  -175.0
   63.0    63.0
  196.0   196.0
   75.0  -169.0
 -175.0    69.0
 -175.0  -175.0
 -123.0  -123.0
  133.0  -111.0
 -249.0  -249.0
  -69.0   175.0
   97.0    97.0
   54.0   175.0


In [48]:
print(value.(B))

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    Dimension 2, [0, 1, 2]
    Dimension 3, [1, 2]
And data, a 24×3×2 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0

  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [49]:
print(value.(ΔB))

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    Dimension 2, [1, 2]
    Dimension 3, [1, 2]
And data, a 24×2×2 Array{Float64, 3}:
[:, :, 1] =
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0

[:, :, 2] =
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0
 -0.0  -0.0